In [3]:
import time
import pandas as pd
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# รายการของค่า list
list_values = [11044959,11044945, 11044964, 11044948, 11044952, 11044946, 11044955, 11044963, 11044960, 11044947,11044950, 11044953, 11044942, 11044944, 11044949, 11044954, 11044956, 11044951, 11044958, 11044943, 11044962, 11044961, 11044957]

# สร้าง DataFrame ว่างๆ เพื่อรวมผลลัพธ์
result_df = pd.DataFrame()

for value in list_values:
    # สร้าง URL จากรายการ
    url = f"https://shopee.co.th/mall/brands/{value}"

    # เปิดเบราว์เซอร์และเข้าไปที่ URL
    options = webdriver.ChromeOptions()
    options.headless = False

    driver = uc.Chrome(options=options)
    driver.get(url)

    try:
        # รอให้ป็อปอัพขึ้น
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="modal"]/div[1]/div[1]/div/div[3]/div[1]/button'))
        )

        # ค้นหา element ด้วย XPath และคลิก
        button = driver.find_element(By.XPATH, '//*[@id="modal"]/div[1]/div[1]/div/div[3]/div[1]/button')
        button.click()

        # รอให้หน้าเว็บโหลดเสร็จ
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'full-brand-list-item__brand-name')))

        # ค้นหา Element ที่มี class "full-brand-list-item__brand-name"
        brand_elements = driver.find_elements(By.CLASS_NAME, 'full-brand-list-item__brand-name')

        # ค้นหา Element ที่มี class "full-brand-list-item__brand-cover-image"
        brand_cover_elements = driver.find_elements(By.CLASS_NAME, 'full-brand-list-item__brand-cover-image')

        # ดึงข้อมูลและบันทึกลงในรายการ
        brand_list = [element.text for element in brand_elements]
        brand_list2 = [element.get_attribute("href") for element in brand_cover_elements]

        # สร้าง DataFrame จากรายการ
        temp_df = pd.DataFrame({'Brand Name': brand_list, 'Brand Link': brand_list2})

        # เพิ่มผลลัพธ์ใน DataFrame รวม
        result_df = pd.concat([result_df, temp_df], ignore_index=True)
        
        driver.close()
        
    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # ปิดเบราว์เซอร์
        driver.quit()

# พิมพ์ผลลัพธ์
driver.quit()
result_df = result_df.drop_duplicates(subset=['Brand Link'], keep='first')
print(result_df)

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")

# driver = uc.Chrome(options=options, use_subprocess=False)

In [3]:
print(result_df['Brand Link'][0])

https://shopee.co.th/above.g


In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import pandas as pd

# Set up the WebDriver (example for Chrome)
options = webdriver.ChromeOptions()
options.add_argument("--headless")

driver = uc.Chrome(options=options, use_subprocess=False)

# Your specified XPath
your_xpath = '//*[@id="main"]/div/div[2]/div/div/div/div[2]/div/div[3]/div[2]/div[2]/div/fieldset/div[2]/section[1]/button[1]/span'

data = []  # List to store data

# Assuming result_df['Brand Link'] contains the URLs
for url in result_df['Brand Link']:
    i = 0
    while True:
        try:
            new_url = url + f"?page={i}"
            driver.get(new_url)

            # Check if the element with your XPath exists
            if not driver.find_elements(by=By.XPATH, value=your_xpath):
                print(f"Element with XPath '{your_xpath}' not found on {new_url}. Moving to next URL.")
                break

            # Find all elements with the specified class and extract their text
            elements = driver.find_elements_by_class_name('shop-search-result-view__item.col-xs-2-4')
            for element in elements:
                data.append(element.text)

            # Increment i to access the next page
            i += 1

        except NoSuchElementException:
            print(f"Element with XPath '{your_xpath}' not found on {new_url}. Moving to next URL.")
            break

        except WebDriverException as e:
            print(f"Error accessing {new_url}: {e}")
            break

# Close the WebDriver
driver.quit()

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['Item Details'])
print(df)
